# 1. 도시명이 "서울특별시"인 행들만 유지한 output csv 생성

In [15]:
import pandas as pd

file_path = "/Users/choiwoojin/2026/quad-s/raw/MOLIT_National_Bus_Stop_Locations_20251031.csv"

df = pd.read_csv(file_path, encoding="cp949")

# 공백 제거 (안전 처리)
df["도시명"] = df["도시명"].astype(str).str.strip()

# 서울만 필터링
seoul_df = df[df["도시명"] == "서울특별시"]

print("서울 정류장 수:", len(seoul_df))

# UTF-8 저장
seoul_df.to_csv(
    "kr_molit_bus_stops_seoul_20251031.csv",
    index=False,
    encoding="utf-8"
)


서울 정류장 수: 16980


# 2. 카카오 `coord2regioncode` API 활용하여 좌표 -> 법정동 코드, 법정동 명 매핑

In [16]:
import pandas as pd
import requests
import time
import os
from dotenv import load_dotenv
import pickle

load_dotenv()

KAKAO_REST_API_KEY = os.getenv("KAKAO_REST_API_KEY")

HEADERS = {
    "Authorization": f"KakaoAK {KAKAO_REST_API_KEY}"
}

COORD2REGION_URL = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"
OUTPUT_PATH = "/Users/choiwoojin/2026/quad-s/output/kr_molit_bus_stops_seoul_20251031.csv"
INPUT_PATH = "/Users/choiwoojin/2026/quad-s/output/kr_molit_bus_stops_seoul_20251031.csv"

# env 정상인지 확인
print(KAKAO_REST_API_KEY)


df = pd.read_csv(INPUT_PATH, encoding="utf-8")

# 좌표만 따로 추출
coords = df[["경도", "위도"]].drop_duplicates().reset_index(drop=True)

print("총 좌표 수:", len(coords))

8d5e873d4ae52b36838aef3796e62a8e
총 좌표 수: 16980


In [17]:
df = pd.read_csv(INPUT_PATH, encoding="utf-8")

# 좌표 숫자 변환
df["경도"] = pd.to_numeric(df["경도"], errors="coerce")
df["위도"] = pd.to_numeric(df["위도"], errors="coerce")
df = df.dropna(subset=["경도", "위도"])

# coord_key 생성
df["coord_key"] = df["경도"].astype(str) + "_" + df["위도"].astype(str)

# 고유 좌표 추출
coords = df[["경도", "위도"]].drop_duplicates().reset_index(drop=True)
coords["coord_key"] = coords["경도"].astype(str) + "_" + coords["위도"].astype(str)

print("총 고유 좌표 수:", len(coords))

총 고유 좌표 수: 16980


In [19]:
coord_map = {}

for idx, row in coords.iterrows():

    params = {
        "x": row["경도"],
        "y": row["위도"]
    }

    try:
        res = requests.get(
            COORD2REGION_URL,
            headers=HEADERS,
            params=params
        )

        if res.status_code == 429:
            print("Rate limit 발생 → 1초 대기")
            time.sleep(1)
            continue

        if res.status_code != 200:
            print("HTTP 에러:", res.status_code)
            coord_map[row["coord_key"]] = (None, None)
            continue

        data = res.json()

        code = None
        name = None

        for doc in data.get("documents", []):
            if doc.get("region_type") == "B":
                code = doc.get("code")
                name = doc.get("region_3depth_name")
                break

        coord_map[row["coord_key"]] = (code, name)

        # rate limit 보호
        time.sleep(0.1)

        # 진행 로그
        if idx % 500 == 0:
            print(f"진행률: {idx}/{len(coords)}")

        # 중간 저장
        if idx % 1000 == 0 and idx != 0:
            with open("coord_map_backup.pkl", "wb") as f:
                pickle.dump(coord_map, f)
            print("중간 저장 완료")

    except Exception as e:
        print("에러 발생:", e)
        coord_map[row["coord_key"]] = (None, None)

print("API 매핑 완료")


df["법정동코드"] = df["coord_key"].map(lambda k: coord_map.get(k, (None, None))[0])
df["법정동명"] = df["coord_key"].map(lambda k: coord_map.get(k, (None, None))[1])

# 임시 key 제거
df = df.drop(columns=["coord_key"])


df.to_csv(
    OUTPUT_PATH,
    index=False,
    encoding="utf-8"
)

print("저장 완료:", OUTPUT_PATH)

진행률: 0/16980
진행률: 500/16980
진행률: 1000/16980
중간 저장 완료
진행률: 1500/16980
진행률: 2000/16980
중간 저장 완료
진행률: 2500/16980
진행률: 3000/16980
중간 저장 완료
진행률: 3500/16980
진행률: 4000/16980
중간 저장 완료
진행률: 4500/16980
진행률: 5000/16980
중간 저장 완료
진행률: 5500/16980
진행률: 6000/16980
중간 저장 완료
진행률: 6500/16980
진행률: 7000/16980
중간 저장 완료
진행률: 7500/16980
진행률: 8000/16980
중간 저장 완료
진행률: 8500/16980
진행률: 9000/16980
중간 저장 완료
진행률: 9500/16980
진행률: 10000/16980
중간 저장 완료
진행률: 10500/16980
진행률: 11000/16980
중간 저장 완료
진행률: 11500/16980
진행률: 12000/16980
중간 저장 완료
진행률: 12500/16980
진행률: 13000/16980
중간 저장 완료
진행률: 13500/16980
진행률: 14000/16980
중간 저장 완료
진행률: 14500/16980
진행률: 15000/16980
중간 저장 완료
진행률: 15500/16980
진행률: 16000/16980
중간 저장 완료
진행률: 16500/16980
API 매핑 완료
저장 완료: /Users/choiwoojin/2026/quad-s/output/kr_molit_bus_stops_seoul_20251031.csv
